In [1]:
import os
import sys
sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2016,21)

- swap position X with position Y means that the letters at indexes X and Y (counting from 0) should be swapped.
- swap letter X with letter Y means that the letters X and Y should be swapped (regardless of where they appear in the string).
- rotate left/right X steps means that the whole string should be rotated; for example, one right rotation would turn abcd into dabc.
- rotate based on position of letter X means that the whole string should be rotated to the right based on the index of letter X (counting from 0) as - determined before this instruction does any rotations. Once the index is determined, rotate the string to the right one time, plus a number of times equal to that index, plus one additional time if the index was at least 4.
- reverse positions X through Y means that the span of letters at indexes X through Y (including the letters at X and Y) should be reversed in order.
move position X to position Y means that the letter which is at index X should be removed from the string, then inserted such that it ends up at index Y.

In [99]:
import re
import collections

patterns = {
    'swap_position': re.compile(r'swap position (\d+) with position (\d+)'),
    'swap_letter': re.compile(r'swap letter (\w+) with letter (\w+)'),
    'rotate_steps': re.compile(r'rotate (\w+) ([\d]+) steps*'),
    'rotate_letter': re.compile(r'rotate based on position of letter (\w)'),
    'move': re.compile(r'move position (\d+) to position (\d)'),
    'reverse': re.compile(r'reverse positions (\d+) through (\d+)')
}


In [9]:
input_lines = [
    'swap position 4 with position 0',
    'swap letter d with letter b',
    'reverse positions 0 through 4',
    'rotate left 1 step',
    'move position 1 to position 4',
    'move position 3 to position 0',
    'rotate based on position of letter b',
    'rotate based on position of letter d'
]

In [139]:
def swap_position(chars, positions):
    i, j = [int(position) for position in positions]
    chars = list(chars)
    chars[i], chars[j] = chars[j], chars[i]
    return ''.join(chars)
    
def swap_letter(chars, letters):
    return swap_position(chars,(chars.find(letters[0]),chars.find(letters[1])))
    
def rotate_steps(chars, steps):
    l_r, count = steps
    if l_r == 'right':
        count = int(count)
    else:
        count = int(count) * -1
    chars = collections.deque(chars)
    chars.rotate(count)
    return ''.join(chars)
    
def rotate_letter(chars, letters):
    """
    rotate based on position of letter X means that the whole string should be rotated
    to the right based on the index of letter X (counting from 0) as - determined before
    this instruction does any rotations. Once the index is determined, rotate the string
    to the right one time, plus a number of times equal to that index, plus one additional
    time if the index was at least 4.
    """
    letter = letters[0]
    pos = chars.find(letter)
    steps = pos + 1
    if pos >= 4:
        steps += 1
    print(f"rotate_letter({chars}, {letter}) - {pos} - {steps}")
    return rotate_steps(chars, ('right', steps))
    
def move(chars, positions):
    print(f"move({chars}, {positions})")
    from_idx, to_idx = [int(position) for position in positions]
    chars = list(chars)
    chars.insert(to_idx, chars.pop(from_idx))
    return ''.join(chars)
    
def reverse_substring(chars, positions):
    i, j = [int(position) for position in positions]
    return chars[:i] + chars[i:j+1][::-1] + chars[j+1:]
    
handlers = {
    'swap_position': swap_position,
    'swap_letter': swap_letter,
    'rotate_steps': rotate_steps,
    'rotate_letter': rotate_letter,
    'move': move,
    'reverse': reverse_substring
}

def scramble(input_string,instruction):
    print(input_string, instruction)
    for key, pattern in patterns.items():
        match = pattern.match(instruction)
        if match:
            new_string = handlers[key](input_string, match.groups())
            break
    return new_string
    

In [141]:
my_string = 'abcde'
for line in input_lines:
    my_string = scramble(my_string, line)
    print(f"Got: {my_string}")


abcde swap position 4 with position 0
Got: ebcda
ebcda swap letter d with letter b
Got: edcba
edcba reverse positions 0 through 4
Got: abcde
abcde rotate left 1 step
Got: bcdea
bcdea move position 1 to position 4
move(bcdea, ('1', '4'))
Got: bdeac
bdeac move position 3 to position 0
move(bdeac, ('3', '0'))
Got: abdec
abdec rotate based on position of letter b
rotate_letter(abdec, b) - 1 - 2
Got: ecabd
ecabd rotate based on position of letter d
rotate_letter(ecabd, d) - 4 - 6
Got: decab


In [143]:
my_str = 'abcde'
from_idx=1
to_idx=3
chars = list(my_str)
chars.insert(to_idx, chars.pop(from_idx))
chars
chars[::-1]

['e', 'b', 'd', 'c', 'a']